# Markov Test
## CS 521 Term Project
Qiren Sun

# Abstraction

A Markov chain is a stochastic model describing a sequence of possible events in which the probability of each event depends only on the state attained in the previous event. This project collected the random webiste and uesd the 2-grams model to obtain the high frequency vocabulary. An n-gram model is a type of probabilistic language model for predicting the next item in such a sequence in the form of a (n − 1)-order Markov model. Both n-gram model and morkov chain model are uesful in NLP(natural language processing). This project choosed the highest frequecy vocabulary that occurred in the 2-grams model to take the markov test. 

#  1. Module

In [4]:
from random import randint
import requests
from bs4 import BeautifulSoup
import re
import string
from collections import OrderedDict
import datetime
import random


# 2. Url Collect 

note: Collecting random website.

In [5]:

class urlCollect:
    def __init__(self,internal=[],bsObj=0,addressParts=0):
        self.internal=internal
        self.bsObj=bsObj
        self.addressParts=addressParts
    
    def getIn(self,bsObj,inUrl):    
        self.internal=[]       
        for i in self.bsObj.findAll("a",href=re.compile("^\/wiki")):
            
            if i.attrs['href'] is not None:
                if i.attrs['href'] not in self.internal:
                    self.internal.append('https://en.wikipedia.org'+i.attrs['href'])
        return self.internal
    
    def splitAddress(self,address):
        self.addressParts=address.replace("http://","").split('/')
        return self.addressParts
        
    def getRanEx(self,startingPage):    
        headers={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
             'Accept-Encoding': 'gzip, deflate, br',
             'Accept-Language': 'zh,en-US;q=0.9,en;q=0.8,zh-TW;q=0.7',
             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
        html=requests.get(startingPage,headers=headers)   
        self.bsObj=BeautifulSoup(html.text,"lxml")   
        self.internal=self.getIn(self.bsObj,self.splitAddress(startingPage)[0])    
        return self.internal[random.randint(0,len(self.internal)-1)]
       
    def fllowEx(self,startingSite):
        self.internal=self.getRanEx(startingSite)
        print("Casual："+self.internal+'\n')
        return self.internal     
       

# 3. Clean Data, Build ngrams model and Generate Markov Chain

In [6]:
def wordListSum(wordList):
    sum = 0
    for word, value in wordList.items():
        sum += value
    return sum

def retrieveRandomWord(wordList):
    randIndex = randint(1, wordListSum(wordList))
    for word, value in wordList.items():
        randIndex -= value
        if randIndex <= 0:
            return word

def buildWordDict(text):
    text = text.replace("\n", " ");
    text = text.replace("\"", "");
    punctuation = [',', '.', ';',':']
    for symbol in punctuation:
        text = text.replace(symbol, " "+symbol+" ");
    words = text.split(" ")
    words = [word for word in words if word != ""]
    wordDict = {}
    for i in range(1, len(words)):
        if words[i-1] not in wordDict:
            wordDict[words[i-1]] = {}
        if words[i] not in wordDict[words[i-1]]:
            wordDict[words[i-1]][words[i]] = 0
        wordDict[words[i-1]][words[i]] = wordDict[words[i-1]][words[i]] + 1
    return wordDict

def cleanInput(input):
    input = re.sub('\n+', " ", input)
    input = re.sub('\[[0-9]*\]', "", input)
    input=re.sub('\t+'," ",input)
    input = re.sub(' +', " ", input)
    input=isCommon(input)
    input = bytes(input, "utf-8")
    input = input.decode("ascii", "ignore")    
    cleanInput = []
    input = input.split(' ')
    for item in input:
        item = item.strip(string.punctuation)
        if len(item) > 1 or (item.lower() == 'a' or item.lower() == 'i'):
            cleanInput.append(item)
    cleanInput=' '.join(cleanInput)
    return cleanInput

def ngrams(input, n):
    #input=cleanInput(input)
    output={}
    for i in range(len(input)-n+1):
        ngramTemp = " ".join(input[i:i+n])
        if ngramTemp not in output:
            output[ngramTemp] = 0
        output[ngramTemp] += 1
    return output

def isCommon(content):
    b=[]
    commonWords = ["the", "be", "and", "of", "a", "in", "to", "have", "it",
    "i", "that", "for", "you", "he", "with", "on", "do", "say", "this",
    "they", "is", "an", "at", "but","we", "his", "from", "that", "not",
    "by", "she", "or", "as", "what", "go", "their","can", "who", "get",
    "if", "would", "her", "all", "my", "make", "about", "know", "will",
    "as", "up", "one", "time", "has", "been", "there", "year", "so",
    "think", "when", "which", "them", "some", "me", "people", "take",
    "out", "into", "just", "see", "him", "your", "come", "could", "now",
    "than", "like", "other", "how", "then", "its", "our", "two", "more",
    "these", "want", "way", "look", "first", "also", "new", "because",
    "day", "more", "use", "no", "man", "find", "here", "thing", "give",
    "many", "well"]
    for word in content.split(' '):
        if word not in commonWords:
            b.append(word)
    b=' '.join(b)
    return b


# 4. Invoke Functions

In [7]:
ages=set()
random.seed(datetime.datetime.now())    
#web collect
url=urlCollect().fllowEx('https://en.wikipedia.org/wiki/Come_and_Get_Your_Love')
headers={'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
         'Accept-Encoding': 'gzip, deflate, br',
         'Accept-Language': 'zh,en-US;q=0.9,en;q=0.8,zh-TW;q=0.7',
         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
html=requests.get(url,headers=headers)   
bsObj=BeautifulSoup(html.text,"lxml")   
content=bsObj.find("div",{"id":"mw-content-text"}).get_text()

#clean content
content = cleanInput(content)
contents=content.split(' ')
ngrams = ngrams(contents, 2)
sortedNGrams = OrderedDict(sorted(ngrams.items(), key=lambda t: t[1], reverse=True))
abstract=[]
for i in range(10):
    abstract.append(list(sortedNGrams.keys())[i])
print(abstract,'\n')
wordDict = buildWordDict(content)

#generate chain length is 100 markov chain
length = 100
chain = ""
currentWord = abstract[0].split(' ')[0]
for i in range(0, length):
    chain += currentWord+" "
    currentWord = retrieveRandomWord(wordDict[currentWord])
print(chain)

#save result to txt file 
with open('markex.txt',"w") as f:
    x=url+'\n'+chain
    f.write(x)    
f.close()


Casual：https://en.wikipedia.org/wiki/North_America

['North America', 'United States', 'North American', 'Archived original', 'June 2011', 'Central American', 'Central America', 'South America', 'Free Trade', 'Trade Agreement'] 

North America Alternatively geologists physiographically locate southern limit Isthmus Panama Canal Figures listed are common region had 1867 already conducted largest trade blocs Canada-Costa Rica 88 , 720 Liga MX Football soccer compete comes big business Guardian . 026 pop . mw-parser-output q{quotes : 100%} . 7 million 1 , 320 2018 North American Region PDF November 2007 What origin Central Intelligence Agence Retrieved June 2011 Garcia-Castellanos Lombardo 2007 Retrieved 20 agriculture comprises 77 population density PDF Press release Bureau definitions Aruba Curaao Sint Eustatius Netherlands Antilles Leeward Antilles Archived original 21 teams 2017 Population estimates are suitable people whom 


Note: This project does not explore the Markov chain model and n-gram model's power in real life and I will continue to complete my work at github. 